## Lab assignment 02

### Neural Machine Translation in the wild
In the third homework you are supposed to get the best translation you can for the EN-RU translation task.

Basic approach using RNNs as encoder and decoder is implemented for you. 

Your ultimate task is to use the techniques we've covered, e.g.

* Optimization enhancements (e.g. learning rate decay)

* Transformer/CNN/<whatever you select> encoder (with or without positional encoding)

* attention/self-attention mechanism

* pretraining the language models (for decoder and encoder)

* or just fine-tunning BART/ELECTRA/... ;)

to improve the translation quality. 

__Please use at least three different approaches/models and compare them (translation quality/complexity/training and evaluation time).__

Write down some summary on your experiments and illustrate it with convergence plots/metrics and your thoughts. Just like you would approach a real problem.

In [2]:
# Thanks to YSDA NLP course team for the data
# (who thanks tilda and deephack teams for the data in their turn)

import os
path_do_data = 'data.txt'
if not os.path.exists(path_do_data):
    print("Dataset not found locally. Downloading from github.")
    !wget https://raw.githubusercontent.com/neychev/made_nlp_course/master/datasets/Machine_translation_EN_RU/data.txt -nc

In [2]:
# Baseline solution BLEU score is quite low. Try to achieve at least __21__ BLEU on the test set. 
# The checkpoints are:

# * __21__ - minimal score to submit the homework, 30% of points

# * __25__ - good score, 70% of points

# * __27__ - excellent score, 100% of points

In [3]:
import numpy as np
import pandas as pd
import torch
import random
import matplotlib.pyplot as plt
import time

from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
from transformers.modeling_outputs import BaseModelOutput
from transformers import T5Model, T5Tokenizer, T5Config, T5ForConditionalGeneration, AutoModelForSeq2SeqLM, AutoModelForCausalLM
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from nltk.translate.bleu_score import corpus_bleu
from IPython.display import clear_output

import wandb

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
with open('data.txt', 'r') as f:
    texts = f.read()

texts = texts.split(sep='\n')
texts = [row.split('\t') for row in texts]
texts_en = [row[0] for row in texts if len(row) == 2]
texts_ru = [row[1] for row in texts if len(row) == 2]

print('Num texts:', len(texts_en), len(texts_ru))
print('En max len:', max([len(row) for row in texts_en]))
print('Ru max len:', max([len(row) for row in texts_ru]))

Num texts: 50000 50000
En max len: 518
Ru max len: 431


In [6]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MAX_LEN = 518
DEVICE

device(type='cuda')

In [7]:
class TextDataset(Dataset):
    def __init__(self, texts_en, texts_ru):
        self.texts_en = texts_en
        self.texts_ru = texts_ru
        
    def __len__(self):
        return len(self.texts_en)
    
    def __getitem__(self, idx):
        return self.texts_en[idx], self.texts_ru[idx]

In [8]:
train_texts_en, val_texts_en, train_texts_ru, val_texts_ru = train_test_split(texts_en, texts_ru, test_size=0.05, random_state=42)
train_texts_en, test_texts_en, train_texts_ru, test_texts_ru = train_test_split(train_texts_en, train_texts_ru, test_size=0.05, random_state=42)

train_dataset = TextDataset(train_texts_en, train_texts_ru)
val_dataset = TextDataset(val_texts_en, val_texts_ru)
test_dataset = TextDataset(test_texts_en, test_texts_ru)

In [9]:
n_epochs = 10
batch_size = 16
log_each_n_iterations = 200
generate_n = 1

In [10]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

train_loader = DataLoader(train_dataset, batch_size, shuffle=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size)
test_loader = DataLoader(test_dataset, batch_size)
generate_loader = DataLoader(val_dataset, generate_n, shuffle=True)


enc_name = 'distilbert-base-multilingual-cased'
dec_name = 't5-small'

enc_tokenizer = AutoTokenizer.from_pretrained(enc_name)
encoder = AutoModel.from_pretrained(enc_name).to(DEVICE)

dec_tokenizer = AutoTokenizer.from_pretrained(dec_name)
config = T5Config(vocab_size=dec_tokenizer.vocab_size, d_model=encoder.config.dim, decoder_start_token_id=0)
decoder = T5ForConditionalGeneration(config).to(DEVICE)

for p in decoder.encoder.parameters():
    p.requires_grad = False
for p in decoder.decoder.parameters():
    p.requires_grad = True


LR = 1e-5
optimizer = torch.optim.AdamW(list(encoder.parameters()) + list(decoder.parameters()), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.5)

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
def encode(texts):
    encoded_input = enc_tokenizer(texts, padding=True, truncation=True, max_length=MAX_LEN, return_tensors='pt')
    with torch.no_grad():
        model_output = encoder(**encoded_input.to(encoder.device))
        embeddings = model_output.last_hidden_state
    return embeddings


def decode(embeddings, max_length=MAX_LEN, repetition_penalty=None, **kwargs):
    with torch.no_grad():
        out = decoder.generate(
            encoder_outputs=BaseModelOutput(last_hidden_state=embeddings), 
            max_length=max_length, 
            repetition_penalty=repetition_penalty,
            **kwargs
        )
        return [dec_tokenizer.decode(tokens, skip_special_tokens=True) for tokens in out]
    

def calc_bleu(loader):
    original_text = []
    generated_text = []
    encoder.eval()
    decoder.eval()

    for en, ru in tqdm(loader):
        embeds = encode(en)
        generated = decode(embeds, max_length=MAX_LEN, repetition_penalty=None)
        
        original_text.extend(ru)
        generated_text.extend(generated)

    return corpus_bleu([[text] for text in original_text], generated_text) * 100

In [12]:
wandb.init(
    # set the wandb project where this run will be logged
    project="nlp-lab2",
    notes="baseline",
    name='experiment_5',
    entity='naumenko-km',
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": LR,
    "encoder": enc_name,
    "decoder": dec_name,
    "epochs": n_epochs,
    }
)
best_bleu = 0
val_bleu = 0
mean_val_loss = 6
iters = 1

for i in range(1, n_epochs + 1):
    print(f'[EPOCH {i}]')
    tqdm_iterator = tqdm(train_loader)
    for text_en_batch, text_ru_batch in tqdm_iterator:
        encoder.train()
        decoder.train()
        x = enc_tokenizer(text_en_batch, return_tensors='pt', padding=True, truncation=True, max_length=MAX_LEN).to(DEVICE)
        y = dec_tokenizer(text_ru_batch, return_tensors='pt', padding=True, truncation=True, max_length=MAX_LEN).to(DEVICE)

        y.input_ids[y.input_ids == 0] = -100  # не учитываем паддинг в лоссе
        embeds = encoder(**x.to(encoder.device))
        embeds = embeds.last_hidden_state.to(DEVICE)

        loss = decoder(
            encoder_outputs=BaseModelOutput(last_hidden_state=embeds),
            labels=y.input_ids,
            decoder_attention_mask=y.attention_mask,
            return_dict=True
        ).loss
                
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        wandb.log({"batch loss": loss.item(), "val loss": mean_val_loss, 'val bleu': val_bleu}, step=iters)

        if iters % log_each_n_iterations == 0:
            encoder.eval()
            decoder.eval()    
            en, ru = next(iter(generate_loader))
            embeds = encode(en)
            generated = decode(embeds, max_length=MAX_LEN, repetition_penalty=3.0)
            example = wandb.Html(data=f'batches: {iters} <br> True: {ru[0]} <br> Generated: {generated[0]}')
            wandb.log({"texts": example}, step=iters)
        iters += 1

    tqdm_iterator = tqdm(test_loader)
    val_loss = []
    for text_en_batch, text_ru_batch in tqdm_iterator:
        encoder.eval()
        decoder.eval()
        with torch.no_grad():
            x = enc_tokenizer(text_en_batch, return_tensors='pt', padding=True, truncation=True, max_length=MAX_LEN).to(DEVICE)
            y = dec_tokenizer(text_ru_batch, return_tensors='pt', padding=True, truncation=True, max_length=MAX_LEN).to(DEVICE)

            y.input_ids[y.input_ids == 0] = -100  # не учитываем паддинг в лоссе
            embeds = encoder(**x.to(encoder.device))
            embeds = embeds.last_hidden_state.to(DEVICE)

            loss = decoder(
                encoder_outputs=BaseModelOutput(last_hidden_state=embeds),
                labels=y.input_ids,
                decoder_attention_mask=y.attention_mask,
                return_dict=True
            ).loss
            val_loss.append(loss.item())
    
    mean_val_loss = np.mean(val_loss)
    val_bleu = calc_bleu(val_loader)
    print("val loss:", mean_val_loss, 'val bleu:', val_bleu)
    scheduler.step()
    
    if val_bleu > best_bleu:
        best_bleu = val_bleu
        # torch.save(encoder.state_dict(), 'encoder.pt')
        # torch.save(decoder.state_dict(), 'decoder.pt')
        # print(f'best loss improved!')

wandb.finish()

wandb: Currently logged in as: naumenko-km. Use `wandb login --relogin` to force relogin


[EPOCH 1]


100%|██████████| 157/157 [11:27<00:00,  4.38s/it]


val loss: 2.5041274640384135 val bleu: 8.64111539902685
[EPOCH 2]


100%|██████████| 157/157 [09:31<00:00,  3.64s/it]


...

100%|██████████| 157/157 [03:51<00:00,  1.48s/it]


val loss: 1.334025996643425 val bleu: 25.659212332389142
[EPOCH 10]


100%|██████████| 157/157 [03:57<00:00,  1.51s/it]


val loss: 1.3221652283764525 val bleu: 25.64166888834994


batch loss,█▆▄▄▄▄▃▃▃▃▃▂▂▂▁▂▃▁▁▂▁▂▁▂▂▂▁▂▁▂▂▂▂▂▂▁▂▂▂▂
val bleu,▁▁▁▁▃▃▃▃▆▆▆▆▇▇▇▇████████████████████████
val loss,████▃▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch loss,1.90227
val bleu,25.65921
val loss,1.33403


In [19]:
# Дообучим еще 1 эпоху
LR = 5e-6
n_epochs = 1
optimizer = torch.optim.AdamW(list(encoder.parameters()) + list(decoder.parameters()), lr=LR)

wandb.init(
    # set the wandb project where this run will be logged
    project="nlp-lab2",
    notes="baseline",
    name='experiment_5-2',
    entity='naumenko-km',
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": LR,
    "encoder": enc_name,
    "decoder": dec_name,
    "epochs": n_epochs,
    }
)
best_bleu = 0
val_bleu = 0
mean_val_loss = 6
iters = 1

for i in range(1, n_epochs + 1):
    print(f'[EPOCH {i}]')
    tqdm_iterator = tqdm(train_loader)
    for text_en_batch, text_ru_batch in tqdm_iterator:
        encoder.train()
        decoder.train()
        x = enc_tokenizer(text_en_batch, return_tensors='pt', padding=True, truncation=True, max_length=MAX_LEN).to(DEVICE)
        y = dec_tokenizer(text_ru_batch, return_tensors='pt', padding=True, truncation=True, max_length=MAX_LEN).to(DEVICE)

        y.input_ids[y.input_ids == 0] = -100  # не учитываем паддинг в лоссе
        embeds = encoder(**x.to(encoder.device))
        embeds = embeds.last_hidden_state.to(DEVICE)

        loss = decoder(
            encoder_outputs=BaseModelOutput(last_hidden_state=embeds),
            labels=y.input_ids,
            decoder_attention_mask=y.attention_mask,
            return_dict=True
        ).loss
                
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        wandb.log({"batch loss": loss.item(), "val loss": mean_val_loss, 'val bleu': val_bleu}, step=iters)

        if iters % log_each_n_iterations == 0:
            encoder.eval()
            decoder.eval()    
            en, ru = next(iter(generate_loader))
            embeds = encode(en)
            generated = decode(embeds, max_length=MAX_LEN, repetition_penalty=3.0)
            example = wandb.Html(data=f'batches: {iters} <br> True: {ru[0]} <br> Generated: {generated[0]}')
            wandb.log({"texts": example}, step=iters)
        iters += 1

    tqdm_iterator = tqdm(test_loader)
    val_loss = []
    for text_en_batch, text_ru_batch in tqdm_iterator:
        encoder.eval()
        decoder.eval()
        with torch.no_grad():
            x = enc_tokenizer(text_en_batch, return_tensors='pt', padding=True, truncation=True, max_length=MAX_LEN).to(DEVICE)
            y = dec_tokenizer(text_ru_batch, return_tensors='pt', padding=True, truncation=True, max_length=MAX_LEN).to(DEVICE)

            y.input_ids[y.input_ids == 0] = -100  # не учитываем паддинг в лоссе
            embeds = encoder(**x.to(encoder.device))
            embeds = embeds.last_hidden_state.to(DEVICE)

            loss = decoder(
                encoder_outputs=BaseModelOutput(last_hidden_state=embeds),
                labels=y.input_ids,
                decoder_attention_mask=y.attention_mask,
                return_dict=True
            ).loss
            val_loss.append(loss.item())
    
    mean_val_loss = np.mean(val_loss)
    val_bleu = calc_bleu(val_loader)
    print("val loss:", mean_val_loss, 'val bleu:', val_bleu)
    scheduler.step()
    
    if val_bleu > best_bleu:
        best_bleu = val_bleu
        # torch.save(encoder.state_dict(), 'encoder.pt')
        # torch.save(decoder.state_dict(), 'decoder.pt')
        # print(f'best loss improved!')

wandb.finish()

[EPOCH 1]


100%|██████████| 157/157 [03:54<00:00,  1.49s/it]


val loss: 1.2625626773642213 val bleu: 27.054727544870538


batch loss,▄▆▃▂▃▄▄▃▂▂▄▅▁▂▃▁▄▅▅▄▆▄▆▆▅▃▅▄▅▄▁█▅▅▂▅▂▃▆▅
val bleu,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch loss,1.35028
val bleu,0
val loss,6


In [20]:
calc_bleu(test_loader)

100%|██████████| 149/149 [03:47<00:00,  1.52s/it]


27.408179247778424